In [ ]:
import pandas as pd

# Create a sample DataFrame with the data you provided
data = {'place': }
df = pd.DataFrame(data)

# Split the 'place' column into two columns using '-' as the separator
df[['place_name', 'city']] = df['place'].str.split('-', n=1, expand=True)

# Strip leading and trailing whitespaces from the 'city' column
df['city'] = df['city'].str.strip()

# Print the updated DataFrame
print(df)


In [ ]:
import pandas as pd 
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv('/Users/rewanabdelqader/Collage/Semster 8/Graduation Project/DS/places csv.csv')

# Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

# Replace NaN with an empty string
df['place_name'] = df['place_name'].fillna('')

# Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df['place_name'])

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Construct a reverse map of indices and place names
indices = pd.Series(df.index, index=df['place_name']).drop_duplicates()

# Define a function that takes in place name as input and outputs the top 10 most similar places sorted by rating
def get_similar_places(place_name, cosine_sim=cosine_sim, df=df):
    # Get the index of the place that matches the title
    idx = indices[place_name]

    # Get the pairwise similarity scores of all places with that place
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the places based on the rating
    sim_scores = sorted(sim_scores, key=lambda x: df['rating'].iloc[x[0]], reverse=True)

    # Get the indices of the top 10 most similar places
    place_indices = [x[0] for x in sim_scores[1:11]]

    # Return the top 10 most similar places sorted by rating
    return df.iloc[place_indices].sort_values(by='rating', ascending=False)

# Test the function with some sample inputs
similar_places = get_similar_places('Rooftop Lounge & Bar, Alexandria')
print(similar_places)

similar_places = get_similar_places('Wunder Garten, Alexandria')
print(similar_places)

In [ ]:
import pandas as pd 
import numpy as np 
df1=pd.read_csv('/Users/rewanabdelqader/Collage/Semster 8/Graduation Project/DS/places csv.csv')
df1.head(5)

place_id	place_name	rating	popularity	rating_count	average_rating	budget	keywords
0	1	Montaza Palace Gardens	2	174.313947	400	4.0	300	['cafe', 'dinner', 'lunch', 'coffee', 'brunch'...
1	2	Exit Games Egypt - Cairo	2	170.926290	200	5.0	100	['cafe', 'dinner', 'pastries', 'breakfast', 't...
2	3	Roasting House - New Cairo, Cairo	1	142.719149	500	5.0	300	['tea', 'cafe', 'lunch', 'desserts']
3	4	Kharga Oasis	3	184.004065	300	3.5	100	['coffee']
4	5	The Secret Chambers Egypt - Cairo	2	156.373940	500	4.0	300	['dinner', 'tea', 'desserts', 'lunch', 'coffee..
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df1['place_name'] = df1['place_name'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df1['place_name'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
#Construct a reverse map of indices and movie titles
indices = pd.Series(df1.index, index=df1['place_name']).drop_duplicates()
# Function that takes in place title as input and outputs most similar movies
def get_recommendations(place_name, cosine_sim=cosine_sim):
    # Get the index of the place that matches the title
    idx = indices[place_name]

    # Get the pairwsie similarity scores of all places with that place
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar place
    sim_scores = sim_scores[1:11]

    # Get the place indices
    places_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df1['place_name'].iloc[places_indices]
    get_recommendations('Rooftop Lounge & Bar, Alexandria')
    181       Pegasus Lounge Bar, Alexandria
128    Sky Roof Bar & Lounge, Alexandria
303                      9 Lounge, Cairo
155                       The Bar, Cairo
399            The Lodge Bar, Alexandria
191       Coffee Lounge - Zamalek, Cairo
40                     Buddha-Bar, Cairo
126           Mojo Lounge & Grill, Cairo
70                   I Bistro Bar, Cairo
25                       Roof Bar, Cairo
Name: place_name, dtype: object
get_recommendations('Wunder Garten, Alexandria')
95       Escape It Egypt - Alexandria
275         Escape Egypt - Alexandria
278       The Room Egypt - Alexandria
471                    Alexandria Zoo
462        The Key Egypt - Alexandria
309          Rooms Egypt - Alexandria
280            Alexandria City Center
351              Cap d'Or, Alexandria
36     Escape Room Egypt - Alexandria
185       Breakout Egypt - Alexandria
Name: place_name, dtype: object

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load user data
user_data = pd.read_csv('/Users/rewanabdelqader/Collage/Semster8/Graduation_Project/DS/Fake Data/User Data.csv')

# Create user profile vector
tfidf_vectorizer = TfidfVectorizer()
user_profile = tfidf_vectorizer.fit_transform(user_data.values.astype('U'))

# Load hangout data
hangout_data = pd.read_csv('hangout_data.csv')


# Create hangout profile vector
hangout_profile = tfidf_vectorizer.fit_transform(hangout_data['description'])

# Calculate similarity between user profile and hangout profile
cosine_similarities = cosine_similarity(user_profile, hangout_profile)

# Get hangout recommendations for each user
hangout_recommendations = {}
for i, row in user_data.iterrows():
    user_id = row['user_id']
    similarity_scores = list(enumerate(cosine_similarities[i]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:6] # Top 5 similar hangouts
    hangout_indices = [i[0] for i in similarity_scores]
    hangout_recommendations[user_id] = list(hangout_data.iloc[hangout_indices]['hangout_name'])

# Print hangout recommendations for each user
for user_id, recommendations in hangout_recommendations.items():
    print(f"Recommendations for user {user_id}:")
    print(', '.join(recommendations))
    print()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load user data
user_interests = pd.read_csv('user_interests.csv')
user_behavior = pd.read_csv('user_behavior.csv')
user_photos = pd.read_csv('user_photos.csv')

# Combine user data into a single dataframe
user_data = pd.concat([user_interests, user_behavior, user_photos], axis=1)

# Clean user data
user_data = user_data.fillna('')

# Create user profile vector
tfidf_vectorizer = TfidfVectorizer()
user_profile = tfidf_vectorizer.fit_transform(user_data.values.astype('U'))

# Load destination data
destination_data = pd.read_csv('destination_data.csv')

# Clean destination data
destination_data = destination_data.fillna('')

# Create destination profile vector
destination_profile = tfidf_vectorizer.transform(destination_data.values.astype('U'))

# Calculate cosine similarity between user profile and destination profiles
similarity_scores = cosine_similarity(user_profile, destination_profile)

# Create a list of recommended destinations for each user
recommendations = []
for i in range(len(user_data)):
    top_destinations = np.argsort(similarity_scores[i])[::-1][:10]
    recommendations.append(list(destination_data.iloc[top_destinations]['destination_name']))

# Save recommendations to a file
with open('user_recommendations.csv', 'w') as f:
    f.write('user_id,recommendations\n')
    for i in range(len(user_data)):
        f.write(f'{i+1},"{", ".join(recommendations[i])}"\n')
